In [1]:
%load_ext autoreload
%autoreload 2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.datasets import MoleculeNet
from torch_geometric.data import DataLoader
import torch_geometric.nn as tgnn
from torch_scatter import scatter
import tqdm
import numpy as np
import wandb

In [2]:
names = ["ESOL", "FreeSolv", "Lipo", "PCBA", "MUV", "HIV", "BACE", "BBBP", "Tox21", "ToxCast", "SIDER", "ClinTox"]

In [11]:
nodes = {}
edges = {}

In [12]:
for name in names:
    try:
        dataset = MoleculeNet('../../datasets/moleculenet', name)
        print(dataset[0])
        max_nodes = np.max([len(d.x) for d in dataset])
        max_edges = np.max([len(d.edge_attr) for d in dataset])
        nodes[name] = max_nodes
        edges[name] = max_edges
    except:
        continue

Data(edge_attr=[68, 3], edge_index=[2, 68], smiles="OCC3OC(OCC2OC(OC(C#N)c1ccccc1)C(O)C(O)C2O)C(O)C(O)C3O ", x=[32, 9], y=[1, 1])
Data(edge_attr=[26, 3], edge_index=[2, 26], smiles="CN(C)C(=O)c1ccc(cc1)OC", x=[13, 9], y=[1, 1])
Data(edge_attr=[54, 3], edge_index=[2, 54], smiles="Cn1c(CN2CCN(CC2)c3ccc(Cl)cc3)nc4ccccc14", x=[24, 9], y=[1, 1])
Data(edge_attr=[44, 3], edge_index=[2, 44], smiles="CC(=O)N1CCC2(CC1)NC(=O)N(c1ccccc1)N2", x=[20, 9], y=[1, 128])
Data(edge_attr=[60, 3], edge_index=[2, 60], smiles="Cc1cccc(N2CCN(C(=O)C34CC5CC(CC(C5)C3)C4)CC2)c1C", x=[26, 9], y=[1, 17])
Data(edge_attr=[40, 3], edge_index=[2, 40], smiles="CCC1=[O+][Cu-3]2([O+]=C(CC)C1)[O+]=C(CC)CC(CC)=[O+]2", x=[19, 9], y=[1, 1])
Data(edge_attr=[70, 3], edge_index=[2, 70], smiles="O1CC[C@@H](NC(=O)[C@@H](Cc2cc3cc(ccc3nc2N)-c2ccccc2C)C)CC1(C)C", x=[32, 9], y=[1, 1])
Data(edge_attr=[40, 3], edge_index=[2, 40], smiles="[Cl].CC(C)NCC(O)COc1cccc2ccccc12", x=[20, 9], y=[1, 1])
Data(edge_attr=[34, 3], edge_index=[2, 34], s

In [13]:
nodes

{'ESOL': 55,
 'FreeSolv': 24,
 'Lipo': 115,
 'PCBA': 332,
 'MUV': 46,
 'HIV': 222,
 'BACE': 97,
 'BBBP': 132,
 'Tox21': 132,
 'ToxCast': 124,
 'SIDER': 492,
 'ClinTox': 136}

In [14]:
edges

{'ESOL': 124,
 'FreeSolv': 50,
 'Lipo': 236,
 'PCBA': 672,
 'MUV': 104,
 'HIV': 502,
 'BACE': 202,
 'BBBP': 290,
 'Tox21': 290,
 'ToxCast': 268,
 'SIDER': 1010,
 'ClinTox': 286}

In [3]:
dataset = MoleculeNet('../../datasets/moleculenet', "HIV")

In [5]:
class_count = np.zeros(2)
for d in dataset:
    #print(d.y)
    #class_count[np.argmax(d.y.numpy()[0])] += 1
    if len(d.x) <= 40:
        class_count[int(d.y[0][0].item())] += 1

In [6]:
class_count[1]/class_count.sum()

0.02758384487709777